# SBV2学習データ準備用のGoogle Colabスクリプト
公式の下記の実装を使わせていただいております。

https://github.com/litagin02/Style-Bert-VITS2/blob/master/colab.ipynb

# Google Driveのマウント
認証が入るので、最初のセルに置くのをオススメします。

In [ ]:
#Google Driveのフォルダをマウント（認証入る）
from google.colab import drive
drive.mount('/content/drive')

# 必要パッケージのインストール

In [ ]:
# このセルを実行して環境構築してください。
# エラーダイアログ「WARNING: The following packages were previously imported in this runtime: [pydevd_plugins]」が出るが「キャンセル」を選択して続行してください。

import os

os.environ["PATH"] += ":/root/.cargo/bin"

!curl -LsSf https://astral.sh/uv/install.sh | sh
!git clone https://github.com/litagin02/Style-Bert-VITS2.git
%cd Style-Bert-VITS2/
!uv pip install --system -r requirements.txt
!git clone https://github.com/mmorise/ita-corpus.git

# 各種パスやモデル名の指定

In [ ]:
model_name = "amitaro_ITA"

# Google Driveでのパスを特定する
import glob
import os
drive_path = os.path.dirname(glob.glob('/content/drive/MyDrive/colab_SBV2train_sample/SBV2-prepare_ITA.ipynb', recursive=True)[0])
print(drive_path)


# ITAコーパスにおける学習用データを整理するスクリプト

ITAコーパスを利用する場合は、文字起こしテキストが存在するため、スライスや文字起こしを別途行う必要はありません。

本ipynbを実行すれば`SBV2-prepare.ipynb`をスキップすることができます。

準備として、`Data/{model_name}/raw`フォルダに、ITAコーパスの音声ファイルを保存してください。
ただし、`emotion`と`recitation`の両方をフォルダ分けして保存してください。
また、それぞれのフォルダ名を`ITA_emotion`と`ITA_recitation`というフォルダ名にして、`Data/{model_name}/raw`フォルダに格納してください。

その後、本ipynbを最初から実行することで、`Data/{model_name}`フォルダに`esd.list`が出力されるはずです。

In [4]:
import os
import re

def create_esd_list(emotion_file, recitation_file, speaker_name, base_folder, output_file="esd.list"):
    try:
        # テキストファイルの内容を読み込む
        with open(emotion_file, "r", encoding="utf-8") as ef:
            emotion_lines = ef.readlines()
        with open(recitation_file, "r", encoding="utf-8") as rf:
            recitation_lines = rf.readlines()

        # フォルダ名を動的に検出
        emotion_folder = None
        recitation_folder = None

        for folder in os.listdir(base_folder):
            if re.search(r'_emotion$', folder, re.IGNORECASE):  # "_emotion"で終わるフォルダ
                emotion_folder = folder
            elif re.search(r'_recitation$', folder, re.IGNORECASE):  # "_recitation"で終わるフォルダ
                recitation_folder = folder

        # 必要なフォルダが見つからない場合エラーを出す
        if emotion_folder is None:
            raise FileNotFoundError("Emotion用のフォルダが見つかりません")
        if recitation_folder is None:
            raise FileNotFoundError("Recitation用のフォルダが見つかりません")

        # 音声ファイル名を取得
        emotion_files = sorted(f for f in os.listdir(os.path.join(base_folder, emotion_folder)) if f.endswith(".wav"))
        recitation_files = sorted(f for f in os.listdir(os.path.join(base_folder, recitation_folder)) if f.endswith(".wav"))

        # 書き起こしテキストを整理
        emotion_texts = [line.strip().split(":", 1)[-1].strip().split(",")[0] for line in emotion_lines if line.strip()]
        recitation_texts = [line.strip().split(":", 1)[-1].strip().split(",")[0] for line in recitation_lines if line.strip()]

        # 出力用リスト
        esd_list = []

        # Emotionの処理
        for audio_file, text in zip(emotion_files, emotion_texts):
            relative_path = f"{emotion_folder}/{audio_file}"
            entry = f"{relative_path}|{speaker_name}|JP|{text}"
            esd_list.append(entry)

        # Recitationの処理
        for audio_file, text in zip(recitation_files, recitation_texts):
            relative_path = f"{recitation_folder}/{audio_file}"
            entry = f"{relative_path}|{speaker_name}|JP|{text}"
            esd_list.append(entry)

        # ファイルに書き込み
        with open(output_file, "w", encoding="utf-8") as of:
            of.write("\n".join(esd_list))

        print(f"'{output_file}' ファイルが作成されました。")
    except Exception as e:
        print(f"エラーが発生しました: {e}")


In [ ]:
os.makedirs(f"{drive_path}/Data/{model_name}/raw", exist_ok=True)

base_folder = f"{drive_path}/Data/{model_name}/raw"
output_file = f"{drive_path}/Data/{model_name}/esd.list"

# 実行例
create_esd_list(
    "./ita-corpus/emotion_transcript_utf8.txt",    # Emotionテキストファイル
    "./ita-corpus/recitation_transcript_utf8.txt",  # Recitationテキストファイル
    model_name,   # 話者名
    base_folder,  # ベースフォルダ
    output_file   # 出力ファイル
)

# 実行後Colabノードブックのランタイムを削除する

特に、有料版のColabでは、接続時間に応じてコンピューティングユニットが消費されてしまうので、節約のため、学習完了したら即座にColabのランタイムを削除する必要がある。

下記セルにおいて、`FLAG = True`として、実行すると、ランタイムが接続解除される

In [ ]:
# ノートブックの解放
FLAG = False

from google.colab import runtime

if FLAG:
    runtime.unassign()